# 東森新聞雲爬蟲練習
## 練習從東森新聞雲網站中，爬取新聞摘要及詳細內容。
## 學習利用Selenium模擬人為操作，更新動態網頁後爬取新聞內容。

In [1]:
import requests
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

### 以財經新聞為例，先連結到財經新聞網頁，複製其URL。

In [2]:
ETtoday_url = "https://www.ettoday.net/news/focus/%E8%B2%A1%E7%B6%93/"  #財經新聞

### 用Selenium打開一個瀏覽器，連結到網站。然後模擬下拉網頁的動作，讓網頁完成更新，再處理後續動作。

In [10]:
browser = webdriver.Chrome()
browser.get(ETtoday_url)  # 打開瀏覽器並連到東森新聞雲網頁

SCROLL_PAUSE_TIME = 1

#
# 以下是用Selenium模擬下拉網頁動作，讓網頁更新
#
#https://kknews.cc/zh-tw/code/avme63n.html
last_height = 0
while True:
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    new_height=browser.execute_script("return document.body.scrollHeight")
    #要用<=的話要time.sleep(2)
    if(new_height==last_height):
        break
    last_height=new_height
    #提示：可參考以下的Stack Overflow: 
    #https://stackoverflow.com/questions/48850974/selenium-scroll-to-end-of-page-indynamically-loading-webpage/48851166


### 到這裡網頁已經更新完畢，所有的新聞都已經出現在網頁上。接下來做爬取收集新聞的動作。

In [11]:
#
# 這裡先建立一個函數，其功能是連到外部連結，並爬取新聞內容。
#
def getNewsDetailContent(link_url):
    resp = requests.get(link_url)
    resp.encoding = 'utf-8'
    #print(resp.text)

    soup = BeautifulSoup(resp.text, 'lxml')
    news_content = soup.find("div", attrs={'class':'story'}).find_all("p")
    for p in news_content:
        """
        .string屬性說明：
        (1) 若當前tag節點底下沒有其他tag子節點，會直接抓取內容(返回"NavigableString")
        (2) 若當前tag節點底下只有唯一的一個tag子節點，也會直接抓取tag子節點的內容(返回"NavigableString")
        (3) 但若當前tag節點底下還有很多個tag子節點，.string就無法判斷，(返回"None")
        """
        if ((p.string) is not None):
            print(p.string)

### 解析HTML並萃取新聞摘要，若有外部的連結，再連到外部連結並把詳細新聞內容抓取下來。

In [20]:
# 爬取網頁內容，解析後萃取新聞摘要
html = browser.page_source
soup = BeautifulSoup(html, "lxml")
all_news = soup.find("div", attrs={'class':'block block_1 infinite_scroll'})

news_block = all_news.find_all('div', attrs={'class':'piece clearfix'})

for i, news_item in enumerate(news_block):
    print("----------------------------------------------------------------------")
    news_body = news_item.find('h3')
    print("\n[%d] %s\n" % (i, news_body.a.string))
    
    # 
    # 連到外部連結，擷取詳細新聞內容
    #
    externalLink = "https://www.ettoday.net" + news_body.a["href"]
    getNewsDetailContent(externalLink)


----------------------------------------------------------------------

[0] 新一代iPad Pro外觀曝光！搭載三顆鏡頭　預計明年3月推出

▲ 2020 iPad Pro。（圖／翻攝自YouTube／iGeeksBlog）
記者李瑞瑾／綜合報導
爆料大神OnLeaks在最近洩漏了兩款2020 iPad Pro的外觀渲染圖與360°影片，圖片顯示新款iPad Pro將追隨iPhone 11 Pro的腳步，搭載三鏡頭模組。
國外科技媒體iGeeksBLOG報導指出，他們收到了OnLeaks洩漏的兩款iPad Pro渲染圖和影片，兩款尺寸分別為11吋和12.9吋。OnLeaks表示，11吋的iPad Pro尺寸可能約為248 x 178.6 x 5.9mm，厚度中也包括後置鏡頭凸起部分的7.8mm。
除了高度和寬度之外，OnLeaks指出，iPad Pro的厚度可能會比前幾代更多一些，因為蘋果計劃為新一代iPad Pro安裝三顆鏡頭的相機模組。
OnLeaks表示，無法確認兩種型號的機身會使用哪種材料（鋁或玻璃），為了模擬這兩種可能性，11吋的渲染模型採用了普通的鋁製機身，而12.9吋的模型採用了玻璃背蓋。
OnLeaks預期，蘋果很可能會在2020年春季發布這兩款iPad Pro，極有可能在明年3月推出；此外，考慮到蘋果一直以來的作業模式，蘋果將在這款iPad Pro中安裝A13X晶片。
----------------------------------------------------------------------

[1] 外媒回顧近十年14支最重要安卓手機　三星走在前端、佔掉近一半席次！

▲ 外媒回顧10年來安卓手機發展，共整理出14支最重要的安卓陣營智慧型手機。（圖／達志影像／美聯社）
記者李瑞瑾／綜合報導
2010年代充滿了技術上的巨大進步，智慧型手機在過去十年中取得了許多進展。隨著一年的結束，9to5Google回顧了這十年來最重要的Android智慧型手機，也同時紀錄了大螢幕、無邊框等趨勢的開始。
Google Nexus One（2010）– Google智慧型手機野心的開始
追溯到2010年代，2010年1月發布的Nexus One之所以出名，是因為它是Nexus

KeyboardInterrupt: 

In [18]:
html = browser.page_source
soup = BeautifulSoup(html, "lxml")
all_news = soup.find_all('div',class_="piece clearfix")
for block in all_news:
    
    title = block.find('h3')
    href = title.find('a')
    print(href["href"],"\n")

/news/20191228/1612388.htm 

/news/20191228/1612325.htm 

/news/20191228/1611993.htm 

/news/20191228/1611632.htm 

/news/20191228/1612177.htm 

/news/20191228/1612176.htm 

/news/20191227/1611886.htm 

/news/20191227/1612078.htm 

/news/20191227/1612051.htm 

/news/20191227/1612049.htm 

/news/20191227/1612039.htm 

/news/20191227/1612033.htm 

/news/20191227/1611860.htm 

/news/20191227/1611986.htm 

/news/20191227/1612002.htm 

/news/20191227/1611889.htm 

/news/20191227/1605870.htm 

/news/20191227/1605863.htm 

/news/20191227/1611983.htm 

/news/20191227/1611968.htm 

//boba.ettoday.net/videonews/195663?from=et_pc_sidebar 

https://reurl.cc/vney11 

https://www.etwarm.com.tw/actions/etwarm-southchina-pet 

https://lihi.cc/pYu2u/a3 

https://boba.ettoday.net/sub_category/229/710?from=boba_issue 

